<a href="https://colab.research.google.com/github/Rotem2411/alephBERTgimmelDalet/blob/main/Latent_Dirichlet_Allocation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
def read_csv_file(file_path):
    data = []
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            data.append(row)
    return data

In [3]:
file_path = 'sentiments.csv'
df = pd.read_csv(file_path)

Take a peek of the data

In [4]:
num_samples = len(df)
print("Total number of samples: ", num_samples)
print(df.head(10))

Total number of samples:  75151
   id                                               text tag category class  \
0   1                    האריות של הצל חזק פה בתגובות...   ש  ECONOMY    b1   
1   2  זמרת תעשייה רק מפרסומות מפורסמת\n  אבל אין קהל...   ש  ECONOMY    b1   
2   4  את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...   ש  ECONOMY    b1   
3   5  שמעון\n  משתתף בפרסומת למילקי במקום לעורר מודע...   ש  ECONOMY    b1   
4   6  הצחקתם אותי\n  מה כל כך אמייזינג בחברה עם אפס ...   ש  ECONOMY    b1   
5   7  איסו חברת הייטק מצליחה צריכה פרסומות שיבואו לע...   ש  ECONOMY    b1   
6   8           לא מכיר אותה ולא שמעתי אפילו שיר אחד שלה   ש  ECONOMY    b1   
7   9  וואללה לא יודע מה מתלהבים מנגה ארז. זה בכלל לא...   ש  ECONOMY    b1   
8  10  אהובה שגיא\n  נגה גדולה.זמרת מעולה.פרסומת מדלי...   ח  ECONOMY    b1   
9  11  הפרסומת של אמדוקס נראית מאוד קודרת ואפורה, כמו...   ש  ECONOMY    b1   

   total_tags  selected_tag  polarity  
0           2             2       1.0  
1           2     

# **Data Pre-processing**

We will perform the following steps:


*   **Tokenization**: Split the text into sentences and the sentences into words
*   Remove **punctuation**.
*   All **stopwords** are removed.
*   Words are **lemmatized** — words in third person are changed to first person and verbs in past and future tenses are changed into present.
*   Words are **stemmed** — words are reduced to their root form.

In [8]:
def contains_letter(word):
    return any(c.isascii() for c in word)

זמרת תעשייה רק מפרסומות מפורסמת
  אבל אין קהל אמיתי שיבוא להופעות


In [9]:
def cleanwords(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('hebrew')
    tokens = nltk.word_tokenize(text)
    no_stopwords = [word for word in tokens if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    no_ascii = [word for word in no_alpha if contains_letter(word)]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    return lemm_text

Due to large size of dataframe we will create a new DataFrame containing 60% of the original data

In [31]:
text_before = df['text']
text_after = {}
for i in text_before.index:
  text_after[i] = cleanwords(text_before[i])

In [32]:
text_comparison = pd.DataFrame({
    'text_before': text_before,
    'text_after': text_after,
})
display(text_comparison.head(5))

,text_before,text_after
0,האריות של הצל חזק פה בתגובות...,"[האריות, הצל, חזק, בתגובות]"
1,זמרת תעשייה רק מפרסומות מפורסמת\n אבל אין קהל...,"[זמרת, תעשייה, מפרסומות, מפורסמת, קהל, אמיתי, ..."
2,"את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...","[יורקת, לבאר, שממנה, שתית, יפה, יפה, חסרת, עמו..."
3,שמעון\n משתתף בפרסומת למילקי במקום לעורר מודע...,"[שמעון, משתתף, בפרסומת, למילקי, במקום, לעורר, ..."
4,הצחקתם אותי\n מה כל כך אמייזינג בחברה עם אפס ...,"[הצחקתם, אמייזינג, בחברה, אפס, נאמנות, לעובדים]"


In [33]:
def vectorize(data, tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names_out()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

In [36]:
for i in range(len(text_after)):
  if 'a' in text_after[i]:
    print(i)

KeyError: 1054

In [41]:
# prompt: now do a loop that check not just letter 'a'. it will check all the letters a-z

text_before = df['text']
text_after = {}
for i in text_before.index:
  for letter in range(65, 91):
    if chr(letter) in text_before[i]:
      del text_before[i]
  for letter in range(97, 123):
    if chr(letter) in text_before[i]:
      del text_before[i]
      break

KeyError: 13

In [40]:
range(97, 123)

'Z'

In [ ]:
# prompt: if find show the index

import pandas as pd
text_before = df['text']
for i in range(len(text_before)):
  if text_before[i].str.contains('about'):
    print(i)


In [10]:
text_after_list = []
for value in text_after.values():
    text_after_list.extend(value)

In [11]:
tfidf_vect = TfidfVectorizer()
tfidf_vect_fit=tfidf_vect.fit(text_after_list)
text_after_list = vectorize(text_after_list, tfidf_vect_fit)
tfidf_matrix = tfidf_vect.fit_transform(text_after_list)
lda_model = LatentDirichletAllocation(n_components=4, random_state=100)
lda_matrix = lda_model.fit_transform(tfidf_matrix)

In [12]:
tfidf_vect.get_feature_names_out()[:100]

array(['about', 'accidentally', 'act', 'adama', 'after', 'agency',
       'agressive', 'air', 'airbnb', 'all', 'alleged', 'also', 'am',
       'amd', 'america', 'an', 'and', 'android', 'any', 'anywho', 'apple',
       'appreciation', 'arab', 'arabe', 'are', 'arm', 'as', 'asani',
       'asshole', 'assignes', 'at', 'avigdor', 'away', 'ba', 'back',
       'bait', 'balea', 'barbivay', 'basically', 'bb', 'bbb', 'bds', 'be',
       'beast', 'because', 'been', 'bet', 'between', 'beצולעת', 'bit',
       'bitcoin', 'black', 'blessing', 'bnb', 'bold', 'bore', 'boy',
       'boycott', 'bracha', 'bravery', 'breville', 'bully', 'bulבול',
       'but', 'by', 'can', 'carefour', 'carters', 'case', 'cash', 'caste',
       'cave', 'cbd', 'cc', 'change', 'chrismas', 'cia', 'clear', 'click',
       'cloud', 'cn', 'cnc', 'cojones', 'cold', 'collaborate', 'com',
       'come', 'company', 'compatible', 'compensation', 'concept',
       'concussion', 'consumer', 'corruption', 'cost', 'costco',
       'countr

In [17]:
word_array = tfidf_vect.get_feature_names_out()
# Convert the array to a list
word_list = list(word_array)

In [23]:
def contains_only_letters(word):
    return word.isalpha()

print(f"'{word_array[1]}' contains only letters: {contains_only_letters(word_array[1])}")

'accidentally' contains only letters: True


In [21]:
from nltk.corpus import words
nltk.download('words')
def is_english_word(word):
    return word.lower() in words.words()

english_words = [word for word in word_list if is_english_word(word)]
print(english_words)  # Output: ['hello', 'tree']

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


KeyboardInterrupt: 

In [ ]:
#####write a code to ignore english words
#### sort the list of words

In [ ]:
n = 30
for index, topic in enumerate(lda_model.components_):
    print(f'The top {n} words for topic #{index}')
    print([tfidf_vect.get_feature_names_out()[i] for i in topic.argsort()[-n:]])

The top 30 words for topic #0
['בעריי', 'תקיפות', 'מישהם', 'להכשיר', 'ושיוויון', 'במלא', 'ותביאו', 'לפיטוריו', 'עכשווי', 'תוספת', 'מימלא', 'ובעיראק', 'ישראייר', 'הנוזלי', 'ינשק', 'הבאגים', 'בקרית', 'ועצבים', 'הדרגתיות', 'פעילי', 'להפתר', 'לבניינים', 'חיבק', 'בקטן', 'תופעות', 'nלמה', 'בשומר', 'כדיי', 'ביבי', 'קיטש']
The top 30 words for topic #1
['והשקיעה', 'לאספקת', 'שמצוינות', 'ומפנק', 'רגלית', 'איככככס', 'הפטריוטיות', 'הסינטטיים', 'לוד', 'חןקי', 'נפתחו', 'איזולירבנד', 'בפולמוס', 'מזמנם', 'בתעופה', 'ספקו', 'איתריום', 'אמיקרון', 'ברכישה', 'צדקן', 'לתיאוקרטיה', 'למרוץ', 'בפודיום', 'הלוזר', 'שמידי', 'וסואבים', 'נפיחה', 'מהסג', 'מריחואנה', 'העיזים']
The top 30 words for topic #2
['פרשנויות', 'מעידות', 'בריח', 'מקניות', 'בעמדה', 'פרמיטיב', 'עסק', 'גומרים', 'ibm', 'בדעיכה', 'והמפכל', 'סיכויו', 'ויגידו', 'לעוה', 'האלופים', 'האייפד', 'ביביזם', 'תקיא', 'היפריע', 'והישיבה', 'בכבוד', 'פיארברג', 'לעמדות', 'למעשיו', 'ללמוד', 'בחומוס', 'מוצאות', 'מתבכיין', 'ועושה', 'הכת']
The top 30 words for topic

In [ ]:
text_after_list = []
for value in text_after.values():
    text_after_list.extend(value)
# We indicate that we would like to exclude any words that show up in less than 2 documents and words that are common across 90% of the documents
cv = CountVectorizer(max_df=0.9, min_df=2)
dtm = cv.fit_transform(text_after_list)

lda = LatentDirichletAllocation(n_components=4, random_state=20)
lda.fit(dtm)

n = 30
for index, topic in enumerate(lda.components_):
    print(f'The top {n} words for topic #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-n:]])

The top 30 words for topic #0
['רוצה', 'לכל', 'לעבוד', 'פחות', 'בישראל', 'בכלל', 'שום', 'פעם', 'שהם', 'שנים', 'וגם', 'שיש', 'עכשיו', 'זו', 'דבר', 'הכי', 'ממש', 'הזה', 'הכל', 'אנשים', 'ביבי', 'כסף', 'כמה', 'לך', 'אחד', 'ישראל', 'עוד', 'צריך', 'ולא', 'שלא']
The top 30 words for topic #1
['שכל', 'צריכים', 'צודק', 'הזאת', 'חבל', 'קשה', 'ראש', 'משהו', 'ואז', 'שכר', 'סוף', 'לשלם', 'אחת', 'ברור', 'מבין', 'ממשלה', 'שאתה', 'אדם', 'בדיוק', 'בארץ', 'כתבה', 'עובדים', 'ללא', 'כלום', 'באמת', 'היו', 'לעשות', 'אפשר', 'מאוד', 'טוב']
The top 30 words for topic #2
['הזו', 'גדול', 'אפילו', 'עושים', 'העולם', 'בעולם', 'כנראה', 'אלא', 'מעניין', 'מחירים', 'מס', 'בית', 'הממשלה', 'שזה', 'רוצים', 'הזמן', 'עבודה', 'נראה', 'לקנות', 'במקום', 'נכון', 'הכבוד', 'היום', 'שהוא', 'פשוט', 'כדי', 'בכל', 'המדינה', 'הרבה', 'כבר']


In [ ]:
text_after_list = []
for value in text_after.values():
    text_after_list.extend(value)

# We indicate that we would like to exclude any words that show up in less than 2 documents and words that are common across 90% of the documents
cv = CountVectorizer(max_df=0.9, min_df=2)
dtm = cv.fit_transform(text_after_list)

In [ ]:
lda = LatentDirichletAllocation(n_components=3, random_state=20)
lda.fit(dtm)

LatentDirichletAllocation(n_components=3, random_state=20)

In [ ]:
n = 30
for index, topic in enumerate(lda.components_):
    print(f'The top {n} words for topic #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-n:]])

The top 30 words for topic #0
['שאתה', 'ממשלה', 'הזמן', 'אדם', 'מדינה', 'שזה', 'רוצה', 'תמיד', 'בארץ', 'בדיוק', 'פחות', 'נכון', 'כתבה', 'במקום', 'עובדים', 'שום', 'עכשיו', 'לעשות', 'היו', 'בכל', 'אפשר', 'הכל', 'המדינה', 'טוב', 'כסף', 'כמה', 'לך', 'אחד', 'ישראל', 'צריך']
The top 30 words for topic #1
['מקום', 'המחירים', 'גדול', 'וזה', 'משהו', 'קשה', 'מחיר', 'העולם', 'אפילו', 'מחירים', 'סוף', 'אלא', 'מס', 'בית', 'לכל', 'עבודה', 'באמת', 'ללא', 'כלום', 'שנים', 'דבר', 'הכי', 'ממש', 'מאוד', 'הזה', 'ביבי', 'הרבה', 'כבר', 'עוד', 'שלא']
The top 30 words for topic #2
['הזאת', 'יודע', 'חבל', 'מעניין', 'לשלם', 'כנראה', 'שנה', 'בעולם', 'ברור', 'עושים', 'הממשלה', 'רוצים', 'הכסף', 'בישראל', 'לעבוד', 'נראה', 'בכלל', 'לקנות', 'הכבוד', 'שהם', 'היום', 'וגם', 'פעם', 'שיש', 'שהוא', 'זו', 'פשוט', 'כדי', 'אנשים', 'ולא']


In [ ]:
data = {
    'Topic': [1, 2, 3],
    'Top Words': [
        [cv.get_feature_names_out()[i] for i in lda.components_[0].argsort()[-n:]],
        [cv.get_feature_names_out()[i] for i in lda.components_[1].argsort()[-n:]],
        [cv.get_feature_names_out()[i] for i in lda.components_[2].argsort()[-n:]]
    ]
}
print(pd.DataFrame(data).to_string())

In [ ]:
topic_results = lda.transform(dtm)
topic = pd.DataFrame(topic_results.argmax(axis=1), columns=["Topic"])
df = pd.concat([df['text'], df['tag'], topic], axis=1)
df.head()

KeyError: 'text'